In [3]:
import pandas as pd
import xlwings as xw
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pythoncom

In [4]:
phone_list = pd.read_csv('stock.csv')
print(phone_list)

                        Name Capacity  Color Grade  Qty        Cost
0           Galaxy A31 A315G     64GB    NaN     C    8  140.000000
1     Galaxy A5 (2016) A510F     16GB    NaN     A   16   56.050132
2  Galaxy A5 (2017) SM-A520F     32GB    NaN     C   11  102.758576
3       Galaxy A52s 5G A528B    128GB    NaN     C    8   65.000000
4           Galaxy A6 A600FN     32GB    NaN     C   27  500.000000
5           Galaxy J6 J600FN     32GB    NaN     A    6  103.000000
6        Galaxy M11 SM-M115F     32GB    NaN     A   11  103.000000
7        Galaxy M13 SM-M135F     64GB    NaN     A    6   75.000000
8  Galaxy Note 10 Lite N770F    128GB    NaN     C   30   75.000000


In [7]:
main_df = pd.DataFrame(columns=['model',
#                                     'category',
                                    'grade',
#                                     'wesell',
                                    'webuy_cash',
#                                     'webuy_voucher',
                                    'low_margin %',
                                    'mid_margin %',
                                    'high_margin %'])

for i, row in phone_list.iterrows():
    website = f"https://uk.webuy.com/search?stext=+{row[0:2]}+&Grade={row[3]}"

    # Selenium and Chrome webscraping configs
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/110.0"

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--window-size=1920,1080")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument("--disable-extensions")
    options.add_argument("--proxy-server='direct://'")
    options.add_argument("--proxy-bypass-list=*")
    options.add_argument("--start-maximized")
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--no-sandbox')
    # driver = webdriver.Chrome(executable_path="chromedriver.exe", options=options)
    driver = webdriver.Chrome(options=options)
    driver.get(website)

    # Wait for the cookie acceptance button to be visible
    wait = WebDriverWait(driver, 10)
    cookie_button = wait.until(EC.visibility_of_element_located((By.ID, "onetrust-accept-btn-handler")))

    # Click the acceptance button
    cookie_button.click()

    # Extracting the data from the website
    phone_models = driver.find_elements(By.XPATH, "//div[@class='desc']")

    # List to append the extracted data
    model = []
#     device = []
    grade = []
#     wesell = []
    webuy_cash = []
#     webuy_voucher = []
    low_margin = []
    mid_margin = []
    high_margin = []

#     print("============================================================")
#     print("Accessing the CEX website...")

    # Looping through each phone search results to get the values
    for models in phone_models:
        name = models.find_element(By.XPATH, ".//span[@class='ais-Highlight']").text
        model.append(name)
#         device.append(models.find_element(By.XPATH, "./p").text)
        grade.append(str(name)[-1])
#         wesell.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeSell for')]").text)
        webuy_cash.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeBuy for cash')]").text)
#         webuy_voucher.append(models.find_element(By.XPATH, ".//div[starts-with(@class,'priceTxt') and starts-with(text(),'WeBuy for voucher')]").text)
        low_margin.append(0.20)
        mid_margin.append(0.25)
        high_margin.append(0.30)
        
 # Rearranging the data into a data frame
    df = pd.DataFrame({'model': model,
#                        'category': device,
                       'grade': grade,
#                        'wesell': wesell,
                       'webuy_cash': webuy_cash,
#                        'webuy_voucher': webuy_voucher,
                       'low_margin %': low_margin,
                       'mid_margin %': mid_margin,
                       'high_margin %': high_margin})

    first_row = df[:1]

    main_df = main_df.append(first_row, ignore_index=True)
    
print(main_df)


                                               model grade  \
0  Samsung Galaxy Tab A T280 7.0" (2016) 8GB Blac...     C   
1    Samsung Galaxy A5 A510F (2016) 16GB Black, O2 A     A   
2  Samsung Galaxy A5 A520F (2017) 32GB Black, Unl...     C   
3  Samsung Galaxy A52s 5G Dual Sim (6GB+128GB) Aw...     C   
4    Samsung Galaxy A6 (2018) 32GB Black, Unlocked C     C   
5  Samsung Galaxy Xcover 5 Dual Sim 64GB Black, V...     A   
6    Samsung Galaxy Tab 3 SM-T211 7" 8GB, Unlocked A     A   
7  Samsung Galaxy Note 10 Lite Dual Sim 128GB Aur...     C   

               webuy_cash  low_margin %  mid_margin %  high_margin %  
0   WeBuy for cash £16.00           0.2          0.25            0.3  
1   WeBuy for cash £37.00           0.2          0.25            0.3  
2   WeBuy for cash £39.00           0.2          0.25            0.3  
3  WeBuy for cash £110.00           0.2          0.25            0.3  
4   WeBuy for cash £43.00           0.2          0.25            0.3  
5   WeBuy for c

In [ ]:
final_df = pd.concat([phone_list, main_df], axis=1)

# Converting text into numbers and calculations
# final_df['wesell'] = final_df['wesell'].str.replace('WeSell for £', '').astype(float)
final_df['webuy_cash'] = final_df['webuy_cash'].str.replace('WeBuy for cash £', '').astype(float)
# final_df['webuy_voucher'] = final_df['webuy_voucher'].str.replace('WeBuy for voucher £', '').astype(float)
final_df['low_margin_cost'] = final_df['webuy_cash'] * 0.80
final_df['mid_margin_cost'] = final_df['webuy_cash'] * 0.75
final_df['high_margin_cost'] = final_df['webuy_cash'] * 0.70
final_df['low_margin %'] = final_df['low_margin %'].map('{:.1%}'.format)
final_df['mid_margin %'] = final_df['mid_margin %'].map('{:.1%}'.format)
final_df['high_margin %'] = final_df['high_margin %'].map('{:.1%}'.format)

print("============================================================")
print("Data rearranged to export")

# Rename columns
final_df = final_df.rename(columns={'Name': 'Supplier_Name',
                                    'Qty': 'Supplier_Qty',
                                    'Cost': 'Supplier_Cost'})

final_df = final_df[[
            'Supplier_Name',
            'Capacity',
            'Color',
            'Grade',
            'Supplier_Qty',
            'Supplier_Cost',
            'model',
            'grade',
            'webuy_cash',
            'low_margin %',
            'low_margin_cost',
            'mid_margin %',
            'mid_margin_cost',
            'high_margin %',
            'high_margin_cost'
    ]]

print(final_df)